# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Prompt Engineering for Instruction-Tuned LLM: Text Transforming & Translation </center>
***


Large language models excel at translation and text transformation, effortlessly converting input from one language to another or aiding in spelling and grammar corrections. 
They are adept at taking imperfectly structured text and refining it, while also capable of converting between various formats, like translating HTML input into JSON output.
Previously, such tasks were arduous and intricate. However, with the advent of large language models, the process has become remarkably simpler. In this article, we will delve into the expressions and prompts that are now far more accessible to implement, thanks to these advanced language models.

#### <a id="top"></a>

# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Text Translation </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. Tone & Text Transformation </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3. Spell & Grammar Check </a></li>     
</ul>
</div>

***


<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Text Translation </b></div>

Let's start with importing OpenAI and also define the get_completion helper function that will take the prompt and return the response.

In [1]:
import os
import utils
import panel as pn
from openai import OpenAI
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI

# ==============================
# 🔑 Load environment variables
# ==============================
env_values = dotenv_values(".env")

openai_api_key = env_values.get("OPENAI_API_KEY")
openai_api_base = env_values.get("OPENAI_API_BASE")
openai_api_name = env_values.get("OPENAI_API_NAME")

# Initialize Clients
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base
)

# Mock moderation for OpenRouter compatibility
if openai_api_base and "openrouter.ai" in openai_api_base:
    class MockResult(dict):
        def __getattr__(self, name): return self.get(name)
    class MockModeration(dict):
        def __init__(self):
            res = MockResult(flagged=False, categories={}, category_scores={})
            super().__init__(results=[res])
        @property
        def results(self): return self["results"]
    client.moderations.create = lambda *args, **kwargs: MockModeration()

chat = ChatOpenAI(
    model_name=openai_api_name, 
    openai_api_key=openai_api_key, 
    openai_api_base=openai_api_base,
    temperature=0.7
)
pn.extension()

# ==============================
# 🔧 Helper Functions
# ==============================
def get_completion(prompt, model=openai_api_name, temperature=0, max_tokens=500):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model=openai_api_name, temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

C:\Users\Administrator\AppData\Local\Temp\ipykernel_25744\312141790.py:41: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


Large language models are trained on a lot of text from many sources, a lot of which is in many different languages. Therefore this kind of model can do translation. These models know kind of hundreds of languages to varying degrees of proficiency. And so we'll go through some examples of how to use this capability.
In this first example, the prompt is to translate the following English text into Arabic. "Hi, I would like to order a new book". Here is the response:

In [2]:
prompt = f"""
Translate the following English text to Arabic: \ 
```Hi, I would like to order a new book```
"""
response = get_completion(prompt)
print(response)

مرحبًا، أود أن أطلب كتابًا جديدًا.


Let's try another example. So in this example, the prompt is "Tell me what language this is". And then this is in French, "Combien coûte le lampadaire".

In [3]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

The language is French. The phrase translates to "How much does the streetlight cost?" in English.


The model has identified that "This language is French".  The model can also do multiple translations at once. So in this example, we will ask the model to translate the text into French, Spanish, and Arabic. 
The text is "I want to order a book and get it delivered to my home and make sure that the book is packaged in a good way so it will arrive in a good form". Let's run it and see how it performs:

In [4]:
prompt = f"""
Translate the following  text to French and Spanish
and Arabic: \
```I want to order a book and get it delivered to my home and make sure that the book is packaged in a good way so it will arrive in a good form```
"""
response = get_completion(prompt)
print(response)

Here are the translations:

**French:**
Je veux commander un livre et le faire livrer chez moi, en m'assurant que le livre est bien emballé pour qu'il arrive en bon état.

**Spanish:**
Quiero pedir un libro y que me lo envíen a casa, asegurándome de que el libro esté bien empaquetado para que llegue en buen estado.

**Arabic:**
أريد أن أطلب كتابًا وأن يتم توصيله إلى منزلي، مع التأكد من أن الكتاب مُعبأ بشكل جيد حتى يصل في حالة جيدة.


In some languages, the translation can change depending on the speaker's relationship to the listener. You can also explain this to the language model. So it will be able to kind of translate accordingly. 
In this example, we ask the LLM to "Translate the following text to Arabic in both the formal and informal forms". "Would you like to order a pillow?"

In [5]:
prompt = f"""
Translate the following text to Arabic in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Sure! Here’s the translation in both formal and informal Arabic:

**Formal Arabic:**
"هل ترغب في طلب وسادة؟"

**Informal Arabic:**
"تحب تطلب مخدة؟"


So here we have the formal and informal. So formal is when you're speaking to someone who's maybe senior to you or you're in a professional situation. That's when you use a formal tone and then informal is when you're speaking to maybe a group of friends.
So for the last example in this section, we're going to pretend that we're in charge of a multinational IT  company and so the user messages are going to be in all different languages and so users are going to be telling us about their IT issues in a wide variety of languages. 
So we need a universal translator. First, we will define a list of user messages in a variety of different languages. Then we will loop through each of these user messages. So the first thing we'll do is ask the model to tell us what language the issue is in. Then we'll print out the original message's language and the issue. Then we'll ask the model to translate it into English and Arabic.

In [6]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Arabic: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (The language of the text you provided is French. The sentence translates to "The performance of the system is slower than usual." in English.): La performance du système est plus lente que d'habitude.


The translation of the text is:

**English:** "The system's performance is slower than usual."

**Arabic:** "أداء النظام أبطأ من المعتاد." 



Original message (The language of the sentence "Mi monitor tiene píxeles que no se iluminan." is Spanish. It translates to "My monitor has pixels that do not light up." in English.): Mi monitor tiene píxeles que no se iluminan.


Translation to English: "My monitor has pixels that do not light up."

Translation to Arabic: "شاشتي تحتوي على بكسلات لا تضيء." 



Original message (The language is Italian. The phrase translates to "My mouse is not working" in English.): Il mio mouse non funziona


The translation of "Il mio mouse non funziona" is:

**English:** "My mouse is not working."

**Arabic:** "الفأرة الخاصة بي لا تعمل." 



Original message (The language of the phrase "Mój klawisz Ctrl jest zepsuty" is Polish. It translates to "My Ctrl key is broken" in English.): Mój klawisz Ctrl jest zepsuty


The translation of the text "Mój klawisz Ctrl jest zepsuty" is:

**English:** "My Ctrl key is broken."

**Arabic:** "مفتاح Ctrl الخاص بي معطل." 



Original message (The language is Chinese. The phrase "我的屏幕在闪烁" translates to "My screen is flickering" in English.): 我的屏幕在闪烁


The translation of the text "我的屏幕在闪烁" is:

**English:** "My screen is flickering."

**Arabic:** "شاشتي تومض." 



<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Tone & Text Transformation </b></div>



Writing can vary based on an intended audience  the way that you would write an email to a colleague or a professor is going to be quite different from the you will text an old friend or a younger brother. ChatGPT can also help produce text with different tones as you define in the input prompt. Let's look at some examples. 
In this first example, we will ask the LLM to transform the email from slang language to a business letter:


In [7]:
prompt = f"""
Transform the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Subject: Specification for Standing Lamp

Dear [Recipient's Name],

I hope this message finds you well. 

I would like to bring to your attention the specifications for the standing lamp. Please take a moment to review the attached document.

Thank you for your consideration.

Best regards,  
Joe


Next, we will convert between different text formats. ChatGPT is very good at translating between different formats such as JSON to HTML and XML, or Markdown. 
So, in the prompt, we'll describe both the input and the output formats. So, in this example, we have this JSON that contains a list of restaurant employees with their names and email. We are going to ask the model to transfer this from JSON to HTML.
The prompt is "Translate the following Python dictionary from JSON to an HTML table with column headers and title". Then we'll get the response from the model and print it.

In [8]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

To convert the provided JSON dictionary into an HTML table, you can use the following HTML code. This code includes a title and column headers for the table:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Restaurant Employees</title>
    <style>
        table {
            width: 50%;
            border-collapse: collapse;
            margin: 20px 0;
        }
        th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>

<h1>Restaurant Employees</h1>

<table>
    <thead>
        <tr>
            <th>Name</th>
            <th>Email</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Shyam</td>
            <td>shyamjaiswal@gmail.com</td>
        </tr>
        <tr>
            <td>Bob</td>
            <td>b

So, here we have some HTML displaying all of the employee names and emails. And so, now let's see if we can view this HTML. We are going to use this display function from this Python library, "display (HTML(response))".

In [9]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Spell & Grammar Check </b></div>


The next transformation task we're going to do is spell check and grammar checking. This is one  of the popular uses for ChatGPT. It's especially useful when you're working in a non-native language.
Let's have a look at some examples of common grammar and spelling problems and how the language model can help address these. I'm going to define a list of sentences that have some grammatical or spelling errors. And then we're going to loop through each of these sentences and ask the model to proofread these.

In [10]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it's oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they're suitcases.",  # Homonyms
  "Your going to need you're notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball


No errors found


Its going to be a long day Does the car need its oil changed


Their goes my freedom. They're going to bring their suitcases.


You're going to need your notebook


That medicine affects my ability to sleep. Have you heard of the butterfly effect?


This phrase is to check chatGPT for spelling ability


We can see that the model can correct all of these grammatical errors. We could use some of the techniques that we've discussed before.
So we'll go through an example of checking a review. And so here is a review about a stuffed panda. And so we're going to ask the model to proofread and correct the review.

In [11]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Here's a proofread and corrected version of your review:

---

I got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults like pandas too! She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was intended to be asymmetrical. Additionally, it's a bit small for what I paid for it; I think there might be other options that are larger for the same price. On a positive note, it arrived a day earlier than expected, so I got to play with it myself before giving it to my daughter.

--- 

Let me know if you need any further adjustments!


We can also print the differences between our original review and the model's output using the Python redlines package. And we're going to get the diff between the original text of our review and the model output and then display this.

In [12]:
!pip install redlines

In [13]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>Here's a proofread and corrected version of your review: </span>

<span style='color:green;font-weight:700;'>--- </span>

<span style='color:green;font-weight:700;'>I got </span>this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my room.  Yes, adults <span style='color:red;font-weight:700;text-decoration:line-through;'>also </span>like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too! </span>She takes it everywhere with her, and it's super soft and cute.  <span style='color:red;font-weight:700;text-decoration:line-through;'>One </span><span style='color:green;font-weight:700;'>However, one </span>of the ears is a bit lower than the other, and I don't think that was <span style='color:red;font-weight:700;text-decoration:line-through;'>designed </span><span style='color:green;font-weight:700;'>intended </span>to be asymmetrical. <span style='color:red;font-weight:700;text-decoration:line-through;'>It's </span><span style='color:green;font-weight:700;'>Additionally, it's </span>a bit small for what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it though. </span><span style='color:green;font-weight:700;'>it; </span>I think there might be other options that are <span style='color:red;font-weight:700;text-decoration:line-through;'>bigger </span><span style='color:green;font-weight:700;'>larger </span>for the same price.  <span style='color:red;font-weight:700;text-decoration:line-through;'>It </span><span style='color:green;font-weight:700;'>On a positive note, it </span>arrived a day earlier than expected, so I got to play with it myself before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>giving </span>it to my daughter.<span style='color:green;font-weight:700;'></span>

<span style='color:green;font-weight:700;'>--- </span>

<span style='color:green;font-weight:700;'>Let me know if you need any further adjustments!</span>

So, the prompt that we used was, "Proofread and correct this review". But you can also make kind of more dramatic changes, changes to tone, and that kind of thing. 
So, let's try one more thing. In this prompt, we're going to ask the model to proofread and correct this same review, but also make it more compelling and ensure that it follows APA style and targets an advanced reader. We are also going to ask for the output in markdown format.

In [14]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

```markdown
### Review of the Panda Plush Toy

I purchased this delightful panda plush toy for my daughter’s birthday, as she has a penchant for borrowing mine from my room. It is worth noting that adults can appreciate the charm of pandas as well. This plush toy has quickly become a beloved companion for her; she takes it everywhere, and its softness and cuteness are undeniable.

However, I did notice a minor flaw: one of the ears is slightly lower than the other, which appears to be an unintended asymmetry rather than a design choice. Additionally, while the quality is commendable, I found the size to be somewhat smaller than I anticipated for the price point. There may be larger alternatives available that offer similar quality at the same cost.

On a positive note, the item arrived a day earlier than expected, allowing me the opportunity to enjoy it myself before presenting it to my daughter. Overall, while there are some drawbacks, the joy it brings to her makes it a worthwhile purchase.
```